## Set cover

In [5]:
import numpy as np
from random import random
from functools import reduce
import queue
from collections import namedtuple

In [6]:
PROBLEM_SIZE = 20
NUM_SETS = 64
State = namedtuple('state', ['taken', 'not_taken'])

In [7]:
SETS = tuple(np.array([random() < .5 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

In [8]:
def goal_check(state): # function to check if we found a set cover
    return len(state.taken) > 0 and np.all(reduce(np.logical_or, [SETS[i] for i in state.taken]))  

In [9]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable" # check if the problem is solvable

#### Classical path search

In [10]:
frontier = queue.Queue()  # FIFO queue: breadth first
#frontier = queue.LifoQueue() # LIFO queue: depth first
frontier.put(State(set(), set(range(NUM_SETS))))  #initial state

In [11]:
curr_state = frontier.get()
counter = 0
while not goal_check(curr_state):
    counter += 1
#    print("Checking", curr_state.taken, curr_state.not_taken)
    for action in curr_state.not_taken:
        new_state = State(curr_state.taken ^ {action}, curr_state.not_taken ^ {action}) #using XOR for both operations
        frontier.put(new_state)
    curr_state = frontier.get()

print("Found solution", curr_state.taken ,"in", counter, "steps.\nCost:", np.sum([SETS[i] for i in curr_state.taken]))

Found solution {0, 41} in 105 steps.
Cost: 25


## Homework
Add a cost equal to the number of true elements in the set, then implement Dijkstra (breadth first with minimum cost)

In [12]:
def cost(state):
    return np.sum([SETS[i] for i in state.taken])

In [13]:
frontier = queue.PriorityQueue()
frontier.put((0, State(set(), set(range(NUM_SETS)))))  #initial state

In [14]:
curr_state = frontier.get()[1]
counter = 0
while not goal_check(curr_state):
    counter += 1
    #print("Checking", curr_state.taken, cost(curr_state))
    for action in curr_state.not_taken:
        new_state = State(curr_state.taken ^ {action}, curr_state.not_taken ^ {action}) #using XOR for both operations
        frontier.put((cost(new_state), new_state))
    curr_state = frontier.get()[1]

print("Found solution", curr_state.taken ,"in", counter, "steps.\nCost:", cost(curr_state))

Found solution {56, 43} in 3577 steps.
Cost: 21


## Informed Strategy

In [15]:
def distance(state):
    return PROBLEM_SIZE - np.sum(reduce(np.logical_or, [SETS[i] for i in state.taken], [False for _ in range(PROBLEM_SIZE)]))

In [16]:
frontier = queue.PriorityQueue()
frontier.put((20, State(set(), set(range(NUM_SETS)))))  #initial state

In [17]:
SETS = tuple(np.array([random() < .5 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

Greedy Best first

In [18]:
_, curr_state = frontier.get()
counter = 0
while not goal_check(curr_state):
    counter += 1
    for action in curr_state.not_taken:
        new_state = State(curr_state.taken ^ {action}, curr_state.not_taken ^ {action}) #using XOR for both operations
        dist = distance(new_state)
        frontier.put((dist, new_state))
    _, curr_state = frontier.get()

print("Found solution", curr_state.taken ,"in", counter, "steps.\nCost:", cost(curr_state))

Found solution {0, 1, 4} in 3 steps.
Cost: 39


Greedy Best First with Mask Optimization

In [19]:
frontier = queue.PriorityQueue()
frontier.put((20, State(set(), set(range(NUM_SETS)))))  #initial state

In [20]:
_, curr_state = frontier.get()
counter = 0
while not goal_check(curr_state):
    counter += 1
    for action in curr_state.not_taken:
        new_state = State(curr_state.taken ^ {action}, curr_state.not_taken ^ {action}) #using XOR for both operations
        dist = distance(new_state, np.logical_not(reduce(np.logical_or, [SETS[i] for i in curr_state.taken], [False for _ in range(PROBLEM_SIZE)])))
        frontier.put((dist, new_state))
    _, curr_state = frontier.get()

print("Found solution", curr_state.taken ,"in", counter, "steps.\nCost:", cost(curr_state))

Found solution {0, 1, 4} in 3 steps.
Cost: 39


In [21]:
for set in curr_state.taken:
    print(SETS[set].astype(int))

[1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1 0 0 1]
[1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 0]
[1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 1 1]
